In [1]:
from qiskit import *
import numpy as np
from scipy.stats import norm
from matplotlib import pyplot as plt
from scipy.stats import rv_continuous
from qiskit.circuit import Parameter

In [2]:
def unitary(circ,eta,phi,t):
    theta = np.arccos(-eta);
    circ.u3(theta,phi,t,0);

In [3]:
class PQC:
    def __init__(self,name,layer):
        self.backend = Aer.get_backend('statevector_simulator');
        self.circ = QuantumCircuit(layer);
        self.name = name;
        self.seed = 14256;
        self.layer = layer;
        np.random.seed(self.seed);
        if self.name == "rcz":
            self.theta = Parameter('θ');
            for index in range(layer):
                self.circ.h(index);
            c = QuantumCircuit(1,name="Rz");
            c.rz(self.theta,0);
            temp = c.to_gate().control(1);
            self.circ.append(temp,[0,1]);
        if self.name == "circ4":
            self.theta1 = [Parameter('θ1')] * layer;
            self.theta2 = [Parameter('θ2')] * layer;
            self.theta3 = [Parameter('θ3')] * (layer-1);
            for i in range(self.layer):
                self.circ.rx(self.theta1[i],i);
            for i in range(self.layer):
                self.circ.rz(self.theta2[i],i);
            for i in range(self.layer-1):
                c = QuantumCircuit(1, name="Rx");
                c.rz(self.theta3[i],0);
                temp = c.to_gate().control(1);
                self.circ.append(temp,[i,i+1]);

    def get(self):
        if self.name == "rcz":
            t = np.random.uniform(0,2*np.pi);
            # theta = Parameter('θ');
            self.circ1 = self.circ.assign_parameters({self.theta:t});
            result = execute(self.circ1,self.backend).result();
            out_state = result.get_statevector();
            self.statevector = np.asmatrix(out_state).T;
            return self.statevector;
        if self.name == "circ4":
            self.circ1 = [self.circ.assign_parameters({self.theta1[i]: np.random.uniform(0,2*np.pi)}) for i in range(self.layer)];
            self.circ2 = [self.circ1[-1].assign_parameters({self.theta2[i]: np.random.uniform(0,2*np.pi)}) for i in range(self.layer)];
            self.circ3 = [self.circ2[-1].assign_parameters({self.theta3[i]: np.random.uniform(0,2*np.pi)}) for i in range(self.layer-1)];
            result = execute(self.circ3[-1],self.backend).result();
            out_state = result.get_statevector();
            self.statevector = np.asmatrix(out_state).T;
            return self.statevector;

    def draw(self):
        self.circ.draw('mpl');
        print(self.circ);

In [4]:
pqc = PQC("circ4",5);

In [5]:
print(pqc.get())

[[-0.20068816+8.13545266e-01j]
 [-0.2004165 -1.06259498e-01j]
 [-0.2004165 -1.06259498e-01j]
 [ 0.03365171+5.13693264e-02j]
 [-0.2004165 -1.06259498e-01j]
 [ 0.04109737-4.56316939e-02j]
 [ 0.03365171+5.13693264e-02j]
 [-0.00127236-1.65761365e-02j]
 [-0.2004165 -1.06259498e-01j]
 [ 0.04109737-4.56316939e-02j]
 [ 0.04109737-4.56316939e-02j]
 [-0.01570459+5.45464925e-03j]
 [ 0.03365171+5.13693264e-02j]
 [-0.01570459+5.45464925e-03j]
 [-0.00127236-1.65761365e-02j]
 [-0.00186281+4.09705105e-03j]
 [-0.03816465-2.23609735e-01j]
 [ 0.06123035+4.69995385e-03j]
 [ 0.06123035+4.69995385e-03j]
 [-0.01390658-9.11011250e-03j]
 [ 0.06123035+4.69995385e-03j]
 [-0.00526534+1.57690656e-02j]
 [-0.01390658-9.11011250e-03j]
 [ 0.00210823+3.97633685e-03j]
 [-0.02014885+5.80109413e-02j]
 [-0.01390658-9.11011250e-03j]
 [-0.01390658-9.11011250e-03j]
 [ 0.00210823+3.97633685e-03j]
 [ 0.0123533 -1.11257838e-02j]
 [ 0.00210823+3.97633685e-03j]
 [-0.00438231+1.02528098e-03j]
 [ 0.00117312+3.29111813e-04j]]


In [6]:
pqc.draw()

     ┌────────┐┌────────┐                                        
q_0: ┤ RX(θ1) ├┤ RZ(θ2) ├────■───────────────────────────────────
     ├────────┤├────────┤┌───┴────┐                              
q_1: ┤ RX(θ1) ├┤ RZ(θ2) ├┤ Rx(θ3) ├────■─────────────────────────
     ├────────┤├────────┤└────────┘┌───┴────┐                    
q_2: ┤ RX(θ1) ├┤ RZ(θ2) ├──────────┤ Rx(θ3) ├────■───────────────
     ├────────┤├────────┤          └────────┘┌───┴────┐          
q_3: ┤ RX(θ1) ├┤ RZ(θ2) ├────────────────────┤ Rx(θ3) ├────■─────
     ├────────┤├────────┤                    └────────┘┌───┴────┐
q_4: ┤ RX(θ1) ├┤ RZ(θ2) ├──────────────────────────────┤ Rx(θ3) ├
     └────────┘└────────┘                              └────────┘
